In [73]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-26 21:46:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Translacja raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Łączenie się z raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... połączono.


Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 200 OK
Długość: 1115394 (1,1M) [text/plain]
Zapis do: ‘input.txt.2’

input.txt.2         100%[===================>]   1,06M  2,45MB/s    w 0,4s     

2024-02-26 21:46:14 (2,45 MB/s) - zapisano ‘input.txt.2’ [1115394/1115394]



In [74]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text), "chars\n-----")
print(text[:100])

1115394 chars
-----
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [75]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [76]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

encode = lambda word: [stoi[c] for c in word] 
decode = lambda word: "".join([itos[c] for c in word])

print(encode("Hello there"))
print(decode(encode("Hello there")))

[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
Hello there


Known tokenizers: sentence piece (google), ticktoken (BPE, openAI)

In [77]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [78]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [82]:
# batch dimention
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in paralell
block_size = 8 # max context_length
def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # beginning of the context
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:", xb.shape, xb, sep="\n")
print("targets:", yb.shape, yb, "----", sep="\n")
# each row is a training example


# time dimention
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(context, "→", target.item())

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
tensor([24]) → 43
tensor([24, 43]) → 58
tensor([24, 43, 58]) → 5
tensor([24, 43, 58,  5]) → 57
tensor([24, 43, 58,  5, 57]) → 1
tensor([24, 43, 58,  5, 57,  1]) → 46
tensor([24, 43, 58,  5, 57,  1, 46]) → 43
tensor([24, 43, 58,  5, 57,  1, 46, 43]) → 39
tensor([44]) → 53
tensor([44, 53]) → 56
tensor([44, 53, 56]) → 1
tensor([44, 53, 56,  1]) → 58
tensor([44, 53, 56,  1, 58]) → 46
tensor([44, 53, 56,  1, 58, 46]) → 39
tensor([44, 53, 56,  1, 58, 46, 39]) → 58
tensor([44, 53, 56,  1, 58, 46, 39, 58]) → 1
tensor([52]) → 58
tensor([52, 58]) → 1
tensor([52, 58,  1]) → 58
tensor([52, 58,  1, 58]) → 46

It's multiple examples at once. We want to predict 18 → 47; 18, 47 → 56; ...

We'd like the transformers to predict for little and big contexts, up to blocksize.

In [96]:
import torch.nn as nn
from torch.nn import functional as F
from math import log
torch.manual_seed(1337)

# baseline – context of 1
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C) – (batch, time, channel)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # pytorch expects C in 2nd dimention
            targets = targets.view(B*T) # or -1
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx) # feed all history to the – not really needed here
            # focus only on the last time step (last elem in time dimention)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss.item(), -log(1/65))

idx = torch.zeros((1,1), dtype=torch.long) # start with a single newline token
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist())) # 0th batch, because it's the only one

torch.Size([256, 65])
4.759584426879883 4.174387269895637

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [90]:
# train
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

for i in range(10000):
    xb, yb = get_batch("test")
    # forward
    logits, loss = m(xb, yb)
    # backward
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # update
    optimizer.step()

print(loss.item())


2.26753830909729


In [95]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist())) # 0th batch, because it's the only one


ONISPENTheagse y hofiniral at, toffthuchot

IOSust s.

ysed;
LOSENIEBe!
ANAPEn
SEN:
I'r, sthou se o-
